In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
fpath = '/Users/xyuan/Documents/BioPharmaFinder/ZhangLab/Data/NoAcidDeg_Phe/Sigma_tRNA_std_150.xlsx'
df_control = load_data(fpath)
df_control.shape

In [ ]:
# df = pd.read_excel('~/Documents/BioPharmaFinder/ZhangLab/total_tRNA_acid_deg_oligosdegcoplex4.xlsx')
df1 = pd.read_excel('~/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
df2 = pd.read_excel('~/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0821.xlsx')
df1 = thermo_df(df1)
df2 = thermo_df(df2)

In [ ]:
fpath = '/Users/xyuan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/total_tRNA_acid_deg.xlsx'
df_total = load_data(fpath)
df_total.shape

In [ ]:
df_5p = load_data('~/Studio/MLC-Seq/examples/data/phe5p.xlsx')
df_3p = load_data('~/Studio/MLC-Seq/examples/data/phe3p.xlsx')
df_5p.shape[0], df_3p.shape[0]

In [ ]:
# plotly_zone(df1[(df1.Mass>2000)&(df1.Mass<20000)], y='Vol')
# df_24k = df1[(df1.Mass>23000)&(df1.Mass<26000)]
plotly_zone(df_24k, y='Vol')
# plotly_zone(df1)

In [ ]:
fpath = '~/Studio/tools/NGS_phe_76_5p.xlsx'
df_ngs_5p = pd.read_excel(fpath)
fpath = '~/Studio/tools/NGS_phe_76_3p.xlsx'
df_ngs_3p = pd.read_excel(fpath)

fpath = '~/Studio/tools/NGS_phe_75_5p.xlsx'
df_ngs_75nt_5p = pd.read_excel(fpath)
fpath = '~/Studio/tools/NGS_phe_75_3p.xlsx'
df_ngs_75nt_3p = pd.read_excel(fpath)

In [ ]:
# dftmp = df_ngs_75nt_3p.copy()
dftmp = df_ngs_3p.copy()
dftmp.index -= 76
dftmp.index *= -1
dftmp[['Mass', 'Base']]

In [ ]:
fpath = '~/Studio/tools/phe_76_5p.xlsx'
df_phe_5p = pd.read_excel(fpath)
fpath = '~/Studio/tools/phe_76_3p.xlsx'
df_phe_3p = pd.read_excel(fpath)
fpath = '~/Studio/tools/phe_75_3p.xlsx'
df_phe_75nt_3p = pd.read_excel(fpath)

In [ ]:
# dfm1 = match_dfs(df_phe_3p, df1)
# dfm2 = match_dfs(df_phe_75nt_3p, df1)
# plotly_zone(dfm1)
# plotly_zone(dfm2)
# dfm1.shape, dfm2.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

# find out data points which can generate base calling
def func2(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)

    bcr = base_calling_random(df, base_only=True)
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

def process(dfs, df_ngs, shift_range, delta, full_mass):
    """ShiftSum
    :param dfs: a set consists of two LC-MS datasets
    :param df_ngs: a set contains two opposite mass ladders of an NGS sequence,
    the first item is the mass ladder of 5´, the second is of 3´
    :param shift_range: iterate the shift in between
    """
    # 1
    print('step 1')
    PROCESSES = 8
    params = [(df_ngs[0], dfs[0], shift) for shift in shift_range]
    with multiprocessing.Pool(PROCESSES) as pool:
        shift_hits = pool.starmap(func, params)
    df_shift_hits_5p = pd.DataFrame(shift_hits)
    
    params = [(df_ngs[1], dfs[1], shift) for shift in shift_range]
    with multiprocessing.Pool(PROCESSES) as pool:
        shift_hits = pool.starmap(func, params)
    df_shift_hits_3p = pd.DataFrame(shift_hits)
    
    # 2
    print('step 2')
    df_sh_top_5p = df_shift_hits_5p[df_shift_hits_5p.hit>1]
    shifts_5p = df_sh_top_5p['shift']
    
    df_sh_top_3p = df_shift_hits_3p[df_shift_hits_3p.hit>1]
    shifts_3p = df_sh_top_3p['shift']
    
    # 3
    print('step 3')
    pairs = list()
    for i in shifts_5p:
        for j in shifts_3p:
            if i+j == delta: #144:
                pairs.append((i, j))
#                 print(i, j, '\n')
    
    # 4
    print('step 4')
    PROCESSES = 8
#     params = [(df_ngs, df_sample, shift) for shift in shifts]
#     params = [(df_ngs[0], dfs[0], pair[0]) for pair in pairs]
    params = [(df_ngs[0], dfs[0], shift) for shift in shifts_5p]
    with multiprocessing.Pool(PROCESSES) as pool:
        df_basecallings = pool.starmap(func2, params)
    df_hits_5p = pd.concat(df_basecallings).drop_duplicates()
    shift_5p = df_hits_5p.Shift.unique()
    plotly_zone(df_hits_5p, y='Shift')
    
#     params = [(df_ngs[1], dfs[1], pair[1]) for pair in pairs]
    params = [(df_ngs[1], dfs[1], shift) for shift in shifts_3p]
    with multiprocessing.Pool(PROCESSES) as pool:
        df_basecallings = pool.starmap(func2, params)
    df_hits_3p = pd.concat(df_basecallings).drop_duplicates()
    shift_3p = df_hits_3p.Shift.unique()
    plotly_zone(df_hits_3p, y='Shift')
    return
    # 5
    print('step 5')
    pairs = list()
    # pairs_75nt = list()
    for i in shift_5p:
        for j in shift_3p:
            if i+j == 11:
                dft = df_hits_5p[df_hits_5p.Shift == i]
                mass_5p = dft.sort_values('Mass')['Mass']
                dft = df_hits_3p[df_hits_3p.Shift == j]
                mass_3p = dft.sort_values('Mass')['Mass']
                max_5p, max_3p = mass_5p.max(), mass_3p.max()
                min_5p, min_3p = mass_5p.min(), mass_3p.min()
                if min_5p + min_3p > full_mass or max_5p + max_3p < full_mass:
                    continue
                pairs.append((i, j))
                print(i, j, '\n', mass_5p.to_list(), '\n', mass_3p.to_list())
    
    selected_5p = [i[0] for i in pairs]
    selected_3p = [i[1] for i in pairs]
    
    # 6
    print('step 6')
    target_shifts = selected_5p.copy()
    dft_5p = df_hits_5p[df_hits_5p.Shift.isin(target_shifts)]
#     plotly_zone(dft_5p, y='Shift')
    target_shifts = selected_3p.copy()
    dft_3p = df_hits_3p[df_hits_3p.Shift.isin(target_shifts)]
#     plotly_zone(dft_3p, y='Shift')
    return dft_5p, dft_3p

In [ ]:
dfs = (df_5p, df_3p)
df_ngs = (df_ngs_5p, df_ngs_3p)
shift_range = np.arange(-160, 160, 1)
delta = 11
fullmass = 24581.38
dfts_76 = process(dfs, df_ngs, shift_range, delta, fullmass)
dfts_76[0].shape

In [ ]:
dfs = (df_5p, df_3p)
df_ngs = (df_ngs_75nt_5p, df_ngs_75nt_3p)
shift_range = np.arange(-160, 160, 1)
delta = 11
fullmass = 24252.31
dfts_75 = process(dfs, df_ngs, shift_range, delta, fullmass)
dfts_75[0].shape

In [ ]:
dfts_76

In [ ]:
plotly_zone(dfts_76[0], y='Shift')
plotly_zone(dfts_76[1], y='Shift')

In [ ]:
plotly_zone(dfts_75[0], y='Shift')
plotly_zone(dfts_75[1], y='Shift')

In [ ]:
dfts_tmp = computational_data_seperation(*dfts_75, 24252.31)
plotly_zones(*dfts_tmp)
dfts_tmp[0].shape
# dfts_76_tmp = computational_data_seperation(*dfts_76, 24581.38)
# plotly_zones(*dfts_76_tmp)
# dfts_76_tmp[0].shape

# df_gap_76_5p = find_gap_dots(dfts_76[0], dfts_76_tmp[0], fullmass_dot=pd.DataFrame())
# df_gap_76_5p.shape
# plotly_zones(dfts_76_tmp[0], df_gap_76_5p, y='Shift', names=['MassSum', 'GapFill'])
# df_gap_76_3p = find_gap_dots(dfts_76[1], dfts_76_tmp[1], fullmass_dot=pd.DataFrame())
# df_gap_76_3p.shape

# plotly_zone(df_3p[(df_3p.Mass>13031)&(df_3p.Mass<14345)])
# plotly_zones(dfts_76_tmp[1], df_gap_76_3p, y='Shift', names=['MassSum', 'GapFill'])

# df_gap_75_5p = find_gap_dots(dfts_75[0], dfts_tmp[0], fullmass_dot=pd.DataFrame())
# df_gap_75_5p.shape
# plotly_zone(df_5p[(df_5p.Mass>10254)&(df_5p.Mass<11567)])
plotly_zones(dfts_tmp[0], df_gap_75_5p, y='Shift', names=['MassSum', 'GapFill'])
# df_gap_75_3p = find_gap_dots(dfts_75[1], dfts_tmp[1], fullmass_dot=pd.DataFrame())
# df_gap_75_3p.shape
plotly_zone(df_3p[(df_3p.Mass>13031)&(df_3p.Mass<14345)])
plotly_zones(dfts_tmp[1], df_gap_75_3p, y='Shift', names=['MassSum', 'GapFill'])

In [ ]:
# plotly_zones(dfts_75[0], dfts_tmp[0], y='Shift')
# plotly_zones(dfts_75[1], dfts_tmp[1], y='Shift')

In [ ]:
# plotly_zones(dfts_76[0], dfts_76_tmp[0], y='Shift')
# plotly_zones(dfts_76[1], dfts_76_tmp[1], y='Shift')

In [ ]:
dft = pd.concat([dfts_76_tmp[0], df_gap_76_5p])
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

In [ ]:
dft = df_ngs[1].copy()
dft.head()

In [ ]:
plt.scatter(dft.Mass[:10], [1]*10)